In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2023-10-24 13:32:24 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


## Package Installation

In [12]:
import Pkg
Pkg.add("StochasticPrograms")
Pkg.add("HiGHS")
Pkg.add("Random")

Pkg.add("Distributions")

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


    Updating `~/.julia/environments/v1.9/Project.toml`
  [9a3f8284] + Random
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


## Package Import

In [1]:
using StochasticPrograms
using HiGHS

In [2]:
using Random, Distributions


In [3]:
using BenchmarkTools

### Complete Solution

In [4]:
procurement_c = [1.0, 100.0, 1000.0] # procurement cost
holding_c = 1.0 # holding cost
stockout_c = [[1.5*c, 5*c, 10*c] for c in procurement_c] # stock out cost

models = Vector{StochasticModel}()

for c in procurement_c
    stockout_c = [1.5*c, 5*c, 10*c]
    for v in stockout_c
        name = "newsvendor_c$(c)_v$(v)"

        @stochastic_model name begin
            @stage 1 begin
                @parameters begin
                    c = c
                end
                @decision(name, x >= 0)
                @constraint(name, x <= 10000)
                @objective(name, Min, x * c)
            end
            @stage 2 begin
                @parameters begin
                  h = holding_c
                  v = v
                end
                @uncertain ξ
                @recourse(name, y >= 0)
                @recourse(name, ξ >= w >= 0)

                @objective(name, Min, h * y + v*w)
                @constraint(name, y >= x-ξ)
                @constraint(name, w >= ξ-x)
            end
        end
        push!(models, name)
    end
end

## Scenarios and Distributions

In [5]:
Random.seed!(42)

TaskLocalRNG()

In [6]:
function generate_scenarios(distrib, max_val = 200)
    scenarios = [@scenario ξ = n probability = cdf(distrib, n) - cdf(distrib, n-1) for n in 1:max_val]
    return scenarios
end

generate_scenarios (generic function with 2 methods)

In [7]:
Unif = Uniform(0, 200)
Exp1 = truncated(Exponential(100), 0, 200)
Exp2 = truncated(Exponential(1e6), 0, 200)

N1 = truncated(Normal(100, 10), 0, 200)
N2 = truncated(Normal(100, 50), 0, 200)
N3 = truncated(Normal(100, 100), 0, 200)

Trian = truncated(TriangularDist(0, 200, 100), 0, 200)
distribs = [Unif, Exp1, Exp2, N1, N2, N3, Trian]


7-element Vector{Distribution{Univariate, Continuous}}:
 Uniform{Float64}(a=0.0, b=200.0)
 Truncated(Exponential{Float64}(θ=100.0); lower=0.0, upper=200.0)
 Truncated(Exponential{Float64}(θ=1.0e6); lower=0.0, upper=200.0)
 Truncated(Normal{Float64}(μ=100.0, σ=10.0); lower=0.0, upper=200.0)
 Truncated(Normal{Float64}(μ=100.0, σ=50.0); lower=0.0, upper=200.0)
 Truncated(Normal{Float64}(μ=100.0, σ=100.0); lower=0.0, upper=200.0)
 Truncated(TriangularDist{Float64}(a=0.0, b=200.0, c=100.0); lower=0.0, upper=200.0)

In [ ]:
[typeof(d) for d in distribs]

7-element Vector{DataType}:
 Uniform{Float64}
 Truncated{Exponential{Float64}, Continuous, Float64, Float64, Float64}
 Truncated{Exponential{Float64}, Continuous, Float64, Float64, Float64}
 Truncated{Normal{Float64}, Continuous, Float64, Float64, Float64}
 Truncated{Normal{Float64}, Continuous, Float64, Float64, Float64}
 Truncated{Normal{Float64}, Continuous, Float64, Float64, Float64}
 Truncated{TriangularDist{Float64}, Continuous, Float64, Float64, Float64}

In [8]:
tot_scenarios = [generate_scenarios(d) for d in distribs]

7-element Vector{Vector{Scenario{NamedTuple{(:ξ,), Tuple{Int64}}}}}:
 [Scenario with probability 0.005
  ξ: 1, Scenario with probability 0.005
  ξ: 2, Scenario with probability 0.004999999999999999
  ξ: 3, Scenario with probability 0.005000000000000001
  ξ: 4, Scenario with probability 0.005000000000000001
  ξ: 5, Scenario with probability 0.0049999999999999975
  ξ: 6, Scenario with probability 0.0050000000000000044
  ξ: 7, Scenario with probability 0.0049999999999999975
  ξ: 8, Scenario with probability 0.0049999999999999975
  ξ: 9, Scenario with probability 0.0050000000000000044
  ξ: 10  …  Scenario with probability 0.0050000000000000044
  ξ: 191, Scenario with probability 0.0050000000000000044
  ξ: 192, Scenario with probability 0.0050000000000000044
  ξ: 193, Scenario with probability 0.0050000000000000044
  ξ: 194, Scenario with probability 0.0050000000000000044
  ξ: 195, Scenario with probability 0.0050000000000000044
  ξ: 196, Scenario with probability 0.0050000000000000044
  ξ:

In [11]:
typeof(models[1])

StochasticModel{2, Tuple{StageParameters{NamedTuple{(:c,), Tuple{Float64}}}, StageParameters{NamedTuple{(:h, :v), Tuple{Float64, Float64}}}}}

## VRD

### EGD

## L-Shaped Solution

In [ ]:
itl = instantiate(models[1], tot_scenarios[1], optimizer = LShaped.Optimizer)
set_optimizer_attribute(itl, MasterOptimizer(), HiGHS.Optimizer)
set_optimizer_attribute(itl, SubProblemOptimizer(), HiGHS.Optimizer)
optimize!(itl)
value(itl[1, :x])

In [156]:
opt_order_quantities = Array(ones(length(models),7))
objective_vals =Array(ones(length(models),7))

for i in 1:length(tot_scenarios)
    println("i\t", i)
    println("Distribution: ", distribs[i])
    for model_idx in 1:length(models)

        # newsvendor = instantiate(models[model_idx], tot_scenarios[i], optimizer = HiGHS.Optimizer)
        newsvendor = instantiate(models[model_idx], tot_scenarios[i], optimizer = LShaped.Optimizer)
        set_optimizer_attribute(newsvendor, MasterOptimizer(), HiGHS.Optimizer)
        set_optimizer_attribute(newsvendor, SubProblemOptimizer(), HiGHS.Optimizer)
        optimize!(newsvendor)
        println("TERMINATION", termination_status(newsvendor))

        opt_order_quantities[ model_idx, i] = value(newsvendor[1,:x])
        
        objective_vals[model_idx, i] = objective_value(newsvendor)


    end


end

Excessive output truncated after 556075 bytes.

Distribution: Uniform{Float64}(a=0.0, b=200.0)
i1

In [157]:
println(opt_order_quantities)
objective_vals

9×7 Matrix{Float64}:
   140.5           98.9488     …     138.279         129.418
   233.83         196.471            224.099         191.632
   264.13         244.909            255.599         220.096
 13394.3         9558.47           13150.8         12337.3
 18113.6        15431.3            17663.8         15874.4
 19130.5        17461.1        …   18824.9         17125.5
     1.33844e5  95532.3           131411.0             1.23292e5
     1.80564e5      1.5381e5           1.76117e5       1.58377e5
     1.90581e5      1.73855e5          1.87568e5  170742.0

### Probability Generation

In [17]:
function generate_probs(distrib, max_val=200)
    probability = [cdf(distrib, n) - cdf(distrib, n-1) for n in 1:max_val]
    return probability
end

generate_probs (generic function with 2 methods)

In [18]:
tot_probs = [generate_probs(d) for d in distribs]

7-element Vector{Vector{Float64}}:
 [0.005, 0.005, 0.004999999999999999, 0.005000000000000001, 0.005000000000000001, 0.0049999999999999975, 0.0050000000000000044, 0.0049999999999999975, 0.0049999999999999975, 0.0050000000000000044  …  0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044]
 [0.011507542817379443, 0.011393040853207947, 0.011279678202616005, 0.011167443529244099, 0.01105632560953134, 0.010946313331593162, 0.010837395694110144, 0.010729561805227661, 0.010622800881467082, 0.010517102246646995  …  0.0017211672898407304, 0.0017040413891614703, 0.001687085894040985, 0.001670299108915696, 0.0016536793550935291, 0.0016372249705846054, 0.001620934309937594, 0.0016048057440717356, 0.001588837660118081, 0.0015730284612541778]
 [0.005000497516417494, 0.005000492515922476, 0.005000487515432464, 0.005000482514947

## EGD

In [175]:
function calculate_egd(real_idx, opt_order_quantities, models, tot_scenarios, distribs)

    opt_order_egd = Array(ones(length(models), 6))
    objective_egd =Array(ones(length(models), 6))

    for i in 1:length(tot_scenarios)

        println("i\t", i)
        println("Distribution: ", distribs[i])

        if i == real_idx 
            println("Skipping ", i)
            continue
        end

        for model_idx in 1:length(models)

            # newsvendor = instantiate(models[model_idx], tot_scenarios[real_idx], optimizer = HiGHS.Optimizer)
            newsvendor = instantiate(models[model_idx], tot_scenarios[real_idx], optimizer = LShaped.Optimizer)
            set_optimizer_attribute(newsvendor, MasterOptimizer(), HiGHS.Optimizer)
            set_optimizer_attribute(newsvendor, SubProblemOptimizer(), HiGHS.Optimizer)

            JuMP.fix(newsvendor[1, :x], opt_order_quantities[model_idx, i])
            optimize!(newsvendor)

            opt_order_egd[model_idx, i-1] = value(newsvendor[1,:x])
            
            objective_egd[model_idx, i-1] = objective_value(newsvendor)
        end

    end
    return opt_order_egd, objective_egd
end

calculate_egd (generic function with 2 methods)

In [176]:
real_idx = 1
opt_order_egd,objective_egd = calculate_egd(real_idx, opt_order_quantities, models, tot_scenarios)

([19.00000000000006 39.99999999999982 … 46.99999999999969 63.99999999999978; 86.00000000000006 133.99999999999983 … 128.99999999999994 119.0000000000002; … ; 118.00000000000006 160.0 … 153.99999999999991 136.99999999999994; 150.99999999999997 180.00000000000003 … 175.00000000000003 156.00000000000003], [143.3875 140.5 … 140.76249999999996 143.94999999999993; 268.14999999999986 233.83 … 234.18 237.1299999999999; … ; 203109.51499999996 180563.6 … 181083.90500000003 187446.58000000002; 212306.62500000003 190580.55 … 191326.125 205560.44999999995])

In [177]:
objective_egd

9×6 Matrix{Float64}:
   143.387        140.5          157.075      …    140.762        143.95
   268.15         233.83         246.3             234.18         237.13
   310.665        264.13         344.725           265.165        280.15
 13798.6        13394.3        13717.8           13401.2        13482.4
 20342.0        18113.6        21394.4           18161.4        18786.6
 21281.6        19130.5        30471.6        …  19201.1        20610.5
     1.3796e5       1.33844e5      1.36973e5         1.33933e5      1.34674e5
     2.0311e5       1.80564e5      2.13679e5         1.81084e5      1.87447e5
     2.12307e5      1.90581e5      3.04432e5         1.91326e5      2.0556e5

## VRD

In [164]:
function calculate_metrics(egd, rpr, calc_vrd=true)
    n, m = size(egd)
    if calc_vrd vrd = Array(ones(n+1,m)) end
    pb = Array(ones(n+1, m))
    # print(size(pb))
    for i = 1:n
        for j = 1:m
            if calc_vrd vrd[i, j] = (egd[i, j] - rpr[i]) /10 end
            pb[i, j] = (egd[i, j] / rpr[i] -1) * 100
        end
    end
    for j = 1:m
        if calc_vrd vrd[n+1, j] = mean(vrd[1:n+1, j]) end
        pb[n+1, j] = mean(pb[1:n+1, j])
    end
    return vrd, pb
end

calculate_metrics (generic function with 2 methods)

In [181]:
opt_order_egd, obj_edg = calculate_egd(i, opt_order_quantities, models, tot_scenarios, distribs)


In [180]:
# for i in 1:length(tot_scenarios)
i = 1

opt_order_egd, objective_egd = calculate_egd(i, opt_order_quantities, models, tot_scenarios, distribs)
print(opt_order_egd)
if i == 1 calc_vrd=true else calc_vrd=false end
vrd, pb = calculate_metrics(opt_order_egd,  objective_vals[:, i], calc_vrd)

println("Distribution: ", distribs[i])
if calc_vrd
    println("VRD: ", vrd)
end
println("PB: ", pb)
# end
print("hey")
# end


In [168]:
pb

10×6 Matrix{Float64}:
  2.05516   0.0          11.7972   1.86833   0.186833   2.45552
 14.6773    2.22045e-14   5.33293  1.05632   0.149681   1.41128
 17.6182    0.0          30.5134   4.65112   0.391852   6.0652
  3.01807  -1.11022e-14   2.41517  0.496442  0.0512905  0.657369
 12.3024   -2.22045e-14  18.1125   2.81584   0.263918   3.71533
 11.2442    0.0          59.2826   5.94471   0.368913   7.73579
  3.07579   0.0           2.33799  0.464905  0.0666637  0.620538
 12.4864    0.0          18.3403   2.89928   0.288156   3.81194
 11.3999    0.0          59.7391   5.50059   0.391213   7.86014
  8.88776   0.1          20.8871   2.66975   0.315852   3.53331